# Checkpointing Models

In [4]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

In [6]:
class MiniVGGNet:
    
    @staticmethod
    def build(width, height, depth, classes, batch_norm=True):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # first CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same",
        input_shape=inputShape))
        model.add(Activation("relu"))
        if batch_norm:
            model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        if batch_norm:
            model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # second CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        if batch_norm:
            model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        if batch_norm:
            model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [7]:
# path to output model
model_path = "./models"

In [8]:
# load the training and testing data, then scale it into the range [0, 1]
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...


In [9]:
# convert the labels from integers to vectors
labelBinarizer = LabelBinarizer()
trainY = labelBinarizer.fit_transform(trainY)
testY = labelBinarizer.transform(testY)

In [10]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [13]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
fname = os.path.sep.join([model_path, "minivgg-weights-{epoch:03d}-{val_loss:.4f}.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
callbacks = [checkpoint]

In [15]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, 
              validation_data=(testX, testY), 
              batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 11s 215us/step - loss: 1.0070 - acc: 0.6484 - val_loss: 0.8303 - val_acc: 0.7075

Epoch 00001: val_loss improved from 1.15101 to 0.83026, saving model to ./models/minivgg-weights-001-0.8303.hdf5
Epoch 2/40
50000/50000 [==============================] - 11s 216us/step - loss: 0.8809 - acc: 0.6914 - val_loss: 0.7889 - val_acc: 0.7224

Epoch 00002: val_loss improved from 0.83026 to 0.78886, saving model to ./models/minivgg-weights-002-0.7889.hdf5
Epoch 3/40
50000/50000 [==============================] - 11s 210us/step - loss: 0.8031 - acc: 0.7178 - val_loss: 0.8782 - val_acc: 0.6996

Epoch 00003: val_loss did not improve from 0.78886
Epoch 4/40
50000/50000 [==============================] - 11s 216us/step - loss: 0.7398 - acc: 0.7398 - val_loss: 0.7217 - val_acc: 0.7500

Epoch 00004: val_loss improved from 0.78886 to 0.72170, saving model to ./models/miniv

50000/50000 [==============================] - 11s 214us/step - loss: 0.2529 - acc: 0.9078 - val_loss: 0.5536 - val_acc: 0.8236

Epoch 00038: val_loss did not improve from 0.54179
Epoch 39/40
50000/50000 [==============================] - 11s 216us/step - loss: 0.2472 - acc: 0.9113 - val_loss: 0.5536 - val_acc: 0.8257

Epoch 00039: val_loss did not improve from 0.54179
Epoch 40/40
50000/50000 [==============================] - 11s 219us/step - loss: 0.2471 - acc: 0.9116 - val_loss: 0.5600 - val_acc: 0.8229

Epoch 00040: val_loss did not improve from 0.54179


In [16]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
fname = os.path.sep.join([model_path, "minivgg-weights-best.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
callbacks = [checkpoint]

In [17]:
H = model.fit(trainX, trainY, 
              validation_data=(testX, testY), 
              batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 11s 216us/step - loss: 0.2394 - acc: 0.9126 - val_loss: 0.5534 - val_acc: 0.8235

Epoch 00001: val_loss improved from inf to 0.55336, saving model to ./models/minivgg-weights-best.hdf5
Epoch 2/40
50000/50000 [==============================] - 11s 215us/step - loss: 0.2382 - acc: 0.9141 - val_loss: 0.5533 - val_acc: 0.8250

Epoch 00002: val_loss improved from 0.55336 to 0.55331, saving model to ./models/minivgg-weights-best.hdf5
Epoch 3/40
50000/50000 [==============================] - 11s 212us/step - loss: 0.2361 - acc: 0.9154 - val_loss: 0.5633 - val_acc: 0.8241

Epoch 00003: val_loss did not improve from 0.55331
Epoch 4/40
50000/50000 [==============================] - 11s 216us/step - loss: 0.2346 - acc: 0.9167 - val_loss: 0.5439 - val_acc: 0.8285

Epoch 00004: val_loss improved from 0.55331 to 0.54386, saving model to ./models/minivgg-weights-best.hdf5
Epoch 5/40
50000/50000